# 7.1  데이터 수집 1 – 한 페이지 크롤링

### 7.1.2 다나와 검색 페이지 접속

In [ ]:
# 예제 7-1 selenium으로 다나와 검색 결과 URL에 접속
from selenium import webdriver
driver = webdriver.Chrome('c:/playwithdata/chromedriver.exe')
url = "http://search.danawa.com/dsearch.php?query=무선청소기&tab=main"
driver.get(url)

### 7.1.3 다나와 검색 웹 페이지에서 상품 정보 가져오기

In [ ]:
from bs4 import BeautifulSoup

# 예제 7-2 웹 페이지의 HTML 정보 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# 예제 7-3 1페이지에 대한 무선청소기 정보 가져오기 1
prod_items = soup.select('li.prod_item')
len(prod_items)

In [ ]:
# 예제 7-4 1페이지에 대한 무선청소기 정보 가져오기 2
prod_items = soup.select('ul.product_list > li.prod_item')
len(prod_items)

In [ ]:
# 예제 7-5 1페이지에 대한 무선청소기 정보 가져오기 3
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
len(prod_items)

In [ ]:
# 예제 7-6 상품명 정보 가져오기
title = prod_items[0].select('p.prod_name > a')[0].text.strip()
print(title)

In [ ]:
# 예제 7-7 스펙 목록 정보 가져오기
spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t','').replace('\n','')  # 수정버전(2022.05.19)
print(spec_list)

In [ ]:
# 예제 7-8 가격 정보 가져오기
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
print(price)

In [ ]:
# 예제 7-9 반복문으로 검색 결과의 1페이지에 대한 상품 정보 추출
prod_data = []
for prod_item in prod_items:
    if 'product-pot' in prod_item['class']:
        continue  # 공백부분(‘li.prod_item.product-pot’) 제외하기
    try:
        # ① 상품명 가져오기
        title = prod_item.select('p.prod_name > a')[0].text.strip()
        # ② 스펙 목록 가져오기
        spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t','').replace('\n','')  # 수정버전(2022.05.19)
        # ③ 가격 정보 가져오기 
        price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",", "")
        
        prod_data.append([title, spec_list, price])
    except:
        pass  # 진행시 에러가 발생할 경우(광고 상품 등) 넘어가기
print(len(prod_data))
print(prod_data)

In [ ]:
# 예제 7-10 상품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        if 'product-pot' in prod_item['class']:
            continue  # 공백부분(‘li.prod_item.product-pot’) 제외하기
        try:
            # ① 상품명 가져오기
            title = prod_item.select('p.prod_name > a')[0].text.strip()
            # ② 스펙 목록 가져오기
            spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t','').replace('\n','')  # 수정버전(2022.05.19)
            # ③ 가격 정보 가져오기
            price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",", "")
            prod_data.append([title, spec_list, price])
        except:
            pass  # 진행시 에러가 발생할 경우(광고 상품 등) 넘어가기
    return prod_data

In [ ]:
# 예제 7-11 상품 정보를 가져오는 함수 테스트
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
prod_data = get_prod_items(prod_items)
print(len(prod_data))

# 7.2  데이터 수집 2 - 여러 페이지에 걸친 다나와 검색 페이지 크롤링

### 7.2.1 다나와 검색 결과 페이지 URL 분석

In [ ]:
# 예제 7-12 다나와 검색 URL을 만들어주는 함수 
def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods'.format(keyword, page)

keyword = '무선청소기'
page = 1
url = get_search_page_url(keyword, page)
print(url)


### 7.2.2 주피터 노트북의 진행표시줄 처리 

In [ ]:
# 예제 7-13 tqdm 라이브러리 설치 
! pip install tqdm

In [ ]:
# 예제 7-14 tqdm 사용법 
import time
from tqdm import tqdm_notebook
total_page = 10
for page in tqdm_notebook(range(1, total_page + 1)):
    # 페이지가 로딩 완료되기 위한 시간을 5초로 준다. 
    time.sleep(5)

### 7.2.3 여러 페이지에 걸친 상품 정보 수집

In [ ]:
# 예제 7-15 실전 다나와 크롤링
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

# driver = webdriver.Chrome('c:/playwithdata/chromedriver.exe')
# # 암묵적으로 웹 자원 로드를 위해 3초까지 기다림
# driver.implicitly_wait(3)

keyword = '무선청소기'
total_page = 10
prod_data_total = []
# 진행 정도를 표현하는 tqdm을 적용 
for page in tqdm_notebook(range(1, total_page + 1)):
    # ① 검색 페이지 이동
    url = get_search_page_url(keyword, page)
    driver.get(url)
    # 페이지가 로딩 완료되기 위한 시간으로 5초를 할당
    time.sleep(5)
    
    # ② 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # ③ 상품 정보 추출
    prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item')
    prod_item_list = get_prod_items(prod_items)
    
    # ④ 추출 데이터 저장
    prod_data_total = prod_data_total + prod_item_list


### 7.2.4 수집 데이터 저장 

In [ ]:
# 예제 7-16 데이터 저장
import pandas as pd
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 목록', '가격']
data.to_excel('./files/1_danawa_crawling_result.xlsx', index = False)